In [268]:
import pandas as pd
import ajuste_sen as aj
import numpy as np
from scipy.optimize import curve_fit
import altair as alt

In [269]:
def graph(data: pd.DataFrame) -> alt.Chart:
    return (
        alt.Chart(data)
        .mark_point(color="red")
        .encode(alt.X("freq", title=""), alt.Y("bode",title = "Gain(dB)"))
    )

#V_C/V_G
#V_R/V_G

In [270]:
frequencia_rc = [46.25,92.5,185,370,740,1480,2960,5920,11840]
frequencia_rl = [2375.375,4750.75,9501.5,19003,38006,76012,152024,304048,608096]
frequencia_rlc = [225,450,900,1800,3600,7200,14400,28800,57600]
amplitude_vg_rc = [2.5,2.5,2.5,2.49,2.48,2.46,2.46,2.45,2.46]
amplitude_vc_rc = [2.47,2.45,2.37,2.16,1.68,1.05,0.58,0.3,0.16]
amplitude_vg_rl = [2.27,2.27,2.28,2.29,2.3,2.36,2.44,2.47,2.46]
amplitdue_vl_rl = [0.12,0.17,0.28,0.53,0.98,1.62,2.16,2.38,2.43]
amplitude_vrc_rlc = [2.35,2.02,1.42,0.79,0.4,0.64,1.27,1.95,2.34]
amplitude_vg_rlc = [2.49,2.45,2.39,2.35,2.33,2.34,2.39,2.45,2.49]

In [271]:
'''def bode_freq(list_freq):
    data = pd.DataFrame()
    data["freq"] = list_freq
    lista_bode = []
    for freq in list_freq:
        lista_bode.append(20 * np.log10(2 * np.pi * freq))
    data["bode"] = lista_bode
    return data

graph(bode_freq(frequencia_rc)).display()
graph(bode_freq(frequencia_rl))'''

'def bode_freq(list_freq):\n    data = pd.DataFrame()\n    data["freq"] = list_freq\n    lista_bode = []\n    for freq in list_freq:\n        lista_bode.append(20 * np.log10(2 * np.pi * freq))\n    data["bode"] = lista_bode\n    return data\n\ngraph(bode_freq(frequencia_rc)).display()\ngraph(bode_freq(frequencia_rl))'

In [272]:
def bode_ganho(list_freq,A_vc,A_vg):
    data = pd.DataFrame()
    data["freq"] = np.log10(list_freq)
    lista_bode = []
    for i in range(len(list_freq)):
        lista_bode.append(20 * np.log10(A_vc[i]/A_vg[i]))
    data["bode"] = lista_bode
    return data

rc = graph(bode_ganho(frequencia_rc,amplitude_vc_rc,amplitude_vg_rc))
rl =graph(bode_ganho(frequencia_rl,amplitdue_vl_rl,amplitude_vg_rl))
rlc = graph(bode_ganho(frequencia_rlc,amplitude_vrc_rlc,amplitude_vg_rlc))

In [274]:
data = pd.read_csv("datas/bode rlc 20.csv")
data["Freq(Hz)"] = np.log10(data["Freq(Hz)"])
data  = data.loc[data["Freq(Hz)"]<=6]

bode_real_rlc = alt.Chart(data).mark_line().encode(
    alt.X("Freq(Hz)"),
    alt.Y("Gain(dB)")
)

data = pd.read_csv("datas/RL0.csv")
data["Freq(Hz)"] = np.log10(data["Freq(Hz)"])
data  = data.loc[data["Freq(Hz)"]<=6]
data = data.loc[data["Freq(Hz)"] >= 3.2]


bode_real_rl = alt.Chart(data).mark_line().encode(
    alt.X("Freq(Hz)", scale = alt.Scale(domain = [3.2,6])),
    alt.Y("Gain(dB)",scale = alt.Scale(domain=[-30,0]))
)

data = pd.read_csv("datas/RC0.csv")
data["Freq(Hz)"] = np.log10(data["Freq(Hz)"])
data  = data.loc[data["Freq(Hz)"]<=4.2]



bode_real_rc = alt.Chart(data).mark_line().encode(
    alt.X("Freq(Hz)"),
    y = "Gain(dB)"
)

(bode_real_rlc + rlc).display()
(bode_real_rlc + rlc).save("bode_rlc.png")
(bode_real_rc + rc).display()
(bode_real_rl + rl).display()

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

In [288]:
def bode_ganho_vr(list_freq,A_vc,A_vg):
    data = pd.DataFrame()
    data["freq"] = np.log10(list_freq)
    lista_bode = []
    for i in range(len(list_freq)):
        lista_bode.append(20 * np.log10(1- A_vc[i]/A_vg[i]))
    data["bode"] = lista_bode
    return data


def bode_vr_rc(freq):
    result = pd.DataFrame()
    result["freq"] = 2 * np.pi * np.array(freq)
    def func(w):
        return 1/(np.sqrt(1-1/(w*2160*0.0000000995)))
    result["bode"] = func(result["freq"])
    return result

a = bode_ganho_vr(frequencia_rc,amplitude_vc_rc,amplitude_vg_rc)
b = bode_vr_rc(np.linspace(0,20000,100))

graph1 = alt.Chart(a).mark_point().encode(
    x = "freq",
    y = "bode"
)

graph2 = alt.Chart(b).mark_line().encode(
    x = "freq:Q",
    y = "bode:Q"
)

graph1
graph2 + graph1

C:\Users\pyde1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


alt.LayerChart(...)

In [275]:
""" 
Rlc arctan(
(frequencia*l - 1/frequencia*c)*(Rt - Rl)/(Rt*Rl + (frequencia*l)^2)

)"""

def func(freq):
    l = 0.005
    c = 0.0000004
    Rt = 425
    Rl = 125
    return np.arctan((freq*l - 1/(freq*c))*(Rt - Rl)/(Rt*Rl + (freq*l - 1/(freq*c))**2))


phase = [-0.267,-0.474,-0.704,-0.758,-0.17,0.704,0.756,0.534,0.289]
df2 = pd.DataFrame()
df2["freq"] = np.array(frequencia_rlc) * 2 * np.pi
df2["bode"] = phase
def bode_topless(lista):
    df = pd.DataFrame()
    lista = np.array(lista)
    df["freq"] = lista
    df["bode"] = func(lista)
    return df


df = bode_topless(np.linspace(450*np.pi,57600*2*np.pi,1000))

_1 = alt.Chart(df).mark_line().encode(
    alt.X("freq", scale = alt.Scale(domain = [225*2*np.pi, 57600*2*np.pi])),
    y = "bode"
)
_2 = alt.Chart(df2).mark_line(point=True,color = "purple").encode(
    x = "freq",
    y = "bode",
    tooltip= ["freq"]
)
_1 + _2

alt.LayerChart(...)

In [276]:
def func(freq):
    l = 0.001
    c = 0.0000004
    Rt = 240
    Rl = 25
    return np.arctan((freq*l)*(Rt - Rl)/(Rt*Rl + (freq*l)**2))

phase = [0.549,0.853,1.083,1.139,1.027,0.75,0.424,0.157,-0.061]
df2 = pd.DataFrame()
df2["freq"] = np.array(frequencia_rl) * 2 * np.pi
df2["bode"] = phase
def bode_topless(lista):
    df = pd.DataFrame()
    lista = np.array(lista)
    df["freq"] = lista
    df["bode"] = func(lista)
    return df


df = bode_topless(np.linspace(2*2375.375*np.pi,608096*2*np.pi,1000))

_1 = alt.Chart(df).mark_line().encode(
    alt.X("freq"
          ),
    y = "bode"
)
_2 = alt.Chart(df2).mark_line(point=True,color = "purple").encode(
    x = "freq",
    y = "bode",
    tooltip= ["freq"]
)
_1 + _2



alt.LayerChart(...)

In [277]:
def func(freq):
    l = 0.001
    c = 99.5* 10**(-9)
    Rt = 2160
    Rl = 25
    return -np.arctan(freq*Rt*c)

phase = [-0.061,-0.131,-0.255,-0.474,-0.786,-1.093,-1.3,-1.42,-1.477]
df2 = pd.DataFrame()
df2["freq"] = np.array(frequencia_rc) * 2 * np.pi
df2["bode"] = phase
def bode_topless(lista):
    df = pd.DataFrame()
    lista = np.array(lista)
    df["freq"] = lista
    df["bode"] = func(lista)
    return df


df = bode_topless(np.linspace(2*46.25*np.pi,11840*2*np.pi,1000))

_1 = alt.Chart(df).mark_line().encode(
    alt.X("freq"
          ),
    y = "bode"
)
_2 = alt.Chart(df2).mark_line(point=True,color = "purple").encode(
    x = "freq",
    y = "bode",
    tooltip= ["freq"]
)
_1  + _2



alt.LayerChart(...)